In [1]:
import requests
from parsel import Selector
import pandas as pd
import numpy as np
import re
import xlwt

获取url

In [20]:
url1 = 'http://www.op.gg/summoner/userName=%EC%B1%84%ED%8C%85%EC%95%88%EC%B9%98%EB%8A%94%EC%86%8C%EB%82%98'
headers = {'Accept-Language': 'zh-CN,zh;q=0.8,zh-CN;q=0.7,zh-CN;q=0.5,en-CN;q=0.3,en;q=0.2',} # 语言-中文 # zh-CN,zh;q=0.9
r1 = requests.get(url1,headers=headers)

In [2]:
# 'http://www.op.gg/summoner/userName=%EC%B1%84%ED%8C%85%EC%95%88%EC%B9%98%EB%8A%94%EC%86%8C%EB%82%98
url2 = 'http://www.op.gg/summoner/matches/ajax/detail/gameId=4687736826&summonerId=40021798&gameTime=1601608111'

response = requests.get(url2,headers=headers)
text = response.text

In [12]:
df1 = pd.DataFrame(columns=['日期','召唤师ID','英雄','评分','段位','击杀','死亡','助攻','击杀贡献率','承受伤害','真眼','放置眼睛','摧毁眼','补兵','平均每分钟补兵','装备'])

In [15]:
selector = Selector(text=text)
# result = selector.xpath('/html/body/div/div[2]/div/div/table/thead/tr/th[1]/text()').getall()
time = selector.xpath().getall() # 日期
ids = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[4]/a/text()').getall() # 召唤师ID 
df1['召唤师ID'] = ids
duanwei = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[5]/text()').getall(); duanwei = [x.strip() for x in duanwei if x.strip() != ''] # 段位
hero = selector.xpath('//body/div/div[2]/div/div/table/tbody/tr/td[1]/a/div[1]/text()').getall()
df1['英雄'] = hero
df1['段位'] = duanwei
score = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[6]/div[1]/text()').getall() # 评分
df1['评分'] = score 
kill = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[7]/div/span[1]/text()').getall() # 击杀
df1['击杀'] = kill
dead = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[7]/div/span[2]/text()').getall() # 死亡
df1['死亡'] = dead
assists = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[7]/div/span[3]/text()').getall() # 助攻
df1['助攻'] = assists
kill_contribute = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[7]/div/span[4]/text()').getall() # 击杀贡献率
df1['击杀贡献率'] = kill_contribute
chengshou = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[8]/div[1]/text()').getall() # 承受伤害
df1['承受伤害'] = chengshou
zhenyan = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[9]/div[1]/span/text()').getall() # 真眼
df1['真眼'] = zhenyan
eye = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[9]/div[2]/span[1]/text()').getall() # 放置眼睛
df1['放置眼睛'] = eye
kill_eye = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[9]/div[2]/span[2]/text()').getall() # 摧毁眼
df1['摧毁眼'] = kill_eye
bubing = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[10]/div[1]/text()').getall() # 补兵
df1['补兵'] = bubing
avg_bubing = selector.xpath('/html/body/div/div[2]/div/div/table/tbody/tr/td[10]/div[2]/text()').getall() # 平均每分钟补兵
df1['平均每分钟补兵'] = avg_bubing
zhuangbeis = [] # 装备
for i in range(1,6):
    zhuangbei = selector.xpath(f'/html/body/div/div[2]/div/div/table[1]/tbody/tr[{i}]/td[11]/div/img'); zhuangbei = [zhuangbei[x].attrib.get('alt') for x in range(len(zhuangbei))]
    zhuangbeis.append(zhuangbei)
for i in range(1,6):
    zhuangbei = selector.xpath(f'/html/body/div/div[2]/div/div/table[2]/tbody/tr[{i}]/td[11]/div/img'); zhuangbei = [zhuangbei[x].attrib.get('alt') for x in range(len(zhuangbei))]
    zhuangbeis.append(zhuangbei)
df1['装备'] = zhuangbeis

In [16]:
df1

,召唤师ID,英雄,评分,段位,击杀,死亡,助攻,击杀贡献率,承受伤害,真眼,放置眼睛,摧毁眼,补兵,平均每分钟补兵,装备
0,김치맹,武器大师,4.9,Diamond 1,3,7,0,(43%),"6,449",0,1,1,150,每分钟 7.9 个,"[破败王者之刃, 忍者足具, 净蚀, 腐败药水, 短剑, 短剑, 监视图腾（饰品）]"
1,조토민,含羞蓓蕾,6.5,Diamond 1,1,5,3,(57%),"8,796",3,4,2,112,每分钟 5.9 个,"[附魔：符能回声, 轻灵之靴, 黑暗封印, 红水晶, 锁子甲, 神谕透镜]"
2,livin1avidaloca,雪原双子,5.9,Diamond 2,2,9,3,(71%),"6,986",3,6,3,93,每分钟 4.9 个,"[多兰之戒, 海克斯科技原型腰带-01, 锁子甲, 速度之靴, 巨人腰带, 黑暗封印, 神谕透镜]"
3,아싸바마,戏命师,4.7,Diamond 1,0,5,2,(29%),"7,382",1,4,1,125,每分钟 6.6 个,"[岚切, 轻灵之靴, 多兰之刃, 监视图腾（饰品）]"
4,채팅안치는소나,琴瑟仙女,3.5,Diamond 1,1,7,0,(14%),"5,177",0,8,0,12,每分钟 0.6 个,"[冰霜之牙, 忍者足具, 雅典娜的邪恶圣杯, 仙女护符, 仙女护符, 神谕透镜]"
5,증폭이 어디갔지,暴怒骑士,7.2,Diamond 1,7,2,11,(55%),"10,219",1,4,0,123,每分钟 6.5 个,"[多兰之刃, 黑色切割者, 巨型九头蛇, 忍者足具, 红水晶, 多兰之盾, 神谕透镜]"
6,상 베,蜘蛛女皇,7,Diamond 1,8,2,9,(52%),"8,455",2,3,2,99,每分钟 5.2 个,"[莫雷洛秘典, 复用型药水, 法师之靴, 附魔：符能回声, 神谕透镜]"
7,이게뭔딜이지,刀锋之影,9.4,Diamond 2,12,1,7,(58%),"20,947",2,7,1,187,每分钟 9.9 个,"[黑色切割者, 提亚马特, 德拉克萨的暮刃, 锯齿短匕, 长剑, 水银之靴, 监视图腾（饰品）]"
8,파르돈,战争女神,6.2,Diamond 2,5,2,10,(45%),"11,024",1,7,1,165,每分钟 8.7 个,"[多兰之刃, 复用型药水, 无尽之刃, 速度之靴, 夺萃之镰, 短剑, 监视图腾（饰品）]"
9,DRX Euro fan T,曙光女神,5.9,Diamond 1,1,0,13,(42%),"3,029",1,6,6,24,每分钟 1.3 个,"[钢铁烈阳之匣, 巨神峰圆盾, 布甲, 破损的秒表, 疾行之靴, 神谕透镜]"
